In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('train.csv', index_col=0)
df['Product ID'] = df['Product ID'].str[1:]
df

,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
id,,,,,,,,,,,,,
0,50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,22284,M,300.1,311.4,1530,37.5,210,0,0,0,0,0,0
136425,38017,H,297.5,308.5,1447,49.1,2,0,0,0,0,0,0
136426,54690,L,300.5,311.8,1524,38.5,214,0,0,0,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136429 entries, 0 to 136428
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Product ID               136429 non-null  object 
 1   Type                     136429 non-null  object 
 2   Air temperature [K]      136429 non-null  float64
 3   Process temperature [K]  136429 non-null  float64
 4   Rotational speed [rpm]   136429 non-null  int64  
 5   Torque [Nm]              136429 non-null  float64
 6   Tool wear [min]          136429 non-null  int64  
 7   Machine failure          136429 non-null  int64  
 8   TWF                      136429 non-null  int64  
 9   HDF                      136429 non-null  int64  
 10  PWF                      136429 non-null  int64  
 11  OSF                      136429 non-null  int64  
 12  RNF                      136429 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 14.6+ MB


In [7]:
df.columns = df.columns.str.replace('[', '').str.replace(']', '')

/tmp/ipykernel_27/2968280360.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('[', '').str.replace(']', '')
/tmp/ipykernel_27/2968280360.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('[', '').str.replace(']', '')


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136429 entries, 0 to 136428
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Product ID             136429 non-null  object 
 1   Type                   136429 non-null  object 
 2   Air temperature K      136429 non-null  float64
 3   Process temperature K  136429 non-null  float64
 4   Rotational speed rpm   136429 non-null  int64  
 5   Torque Nm              136429 non-null  float64
 6   Tool wear min          136429 non-null  int64  
 7   Machine failure        136429 non-null  int64  
 8   TWF                    136429 non-null  int64  
 9   HDF                    136429 non-null  int64  
 10  PWF                    136429 non-null  int64  
 11  OSF                    136429 non-null  int64  
 12  RNF                    136429 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 14.6+ MB


In [10]:
test_data = pd.read_csv('test.csv')
test_data.columns = test_data.columns.str.replace('[', '').str.replace(']', '')

/tmp/ipykernel_27/2865127998.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test_data.columns = test_data.columns.str.replace('[', '').str.replace(']', '')
/tmp/ipykernel_27/2865127998.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test_data.columns = test_data.columns.str.replace('[', '').str.replace(']', '')


In [14]:
test_data['Product ID'] = test_data['Product ID'].astype(str)

In [16]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90954 entries, 0 to 90953
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Product ID             90954 non-null  object 
 1   Type                   90954 non-null  object 
 2   Air temperature K      90954 non-null  float64
 3   Process temperature K  90954 non-null  float64
 4   Rotational speed rpm   90954 non-null  int64  
 5   Torque Nm              90954 non-null  float64
 6   Tool wear min          90954 non-null  int64  
 7   TWF                    90954 non-null  int64  
 8   HDF                    90954 non-null  int64  
 9   PWF                    90954 non-null  int64  
 10  OSF                    90954 non-null  int64  
 11  RNF                    90954 non-null  int64  
dtypes: float64(3), int64(7), object(2)
memory usage: 8.3+ MB


In [18]:
df.columns = df.columns.str.replace(' ', '')
df

,ProductID,Type,AirtemperatureK,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,Machinefailure,TWF,HDF,PWF,OSF,RNF
id,,,,,,,,,,,,,
0,50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
136424,22284,M,300.1,311.4,1530,37.5,210,0,0,0,0,0,0
136425,38017,H,297.5,308.5,1447,49.1,2,0,0,0,0,0,0
136426,54690,L,300.5,311.8,1524,38.5,214,0,0,0,0,0,0


In [20]:
test_data.columns = test_data.columns.str.replace(' ', '')
test_data

,ProductID,Type,AirtemperatureK,ProcesstemperatureK,Rotationalspeedrpm,TorqueNm,Toolwearmin,TWF,HDF,PWF,OSF,RNF
0,50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
90949,51130,L,302.3,311.4,1484,40.4,15,0,0,0,0,0
90950,47783,L,297.9,309.8,1542,33.8,31,0,0,0,0,0
90951,48097,L,295.6,306.2,1501,41.4,187,0,0,0,0,0
90952,48969,L,298.1,307.8,1534,40.3,69,0,0,0,0,0


In [21]:
df.to_csv('df.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

In [28]:
filtered_df = df[df["Machinefailure"] == 1]
filtered_df.to_csv('f_data.csv', index=False)

In [23]:
df["Machinefailure"].unique()

array([0, 1])